In [321]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers
import tensorflow.random
import pandas as pd
import numpy as np 
import matplotlib 
import matplotlib.pyplot as plt
from sklearn import metrics
import math
from math import log
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
#import sys
import os
import time

# Utilities

In [322]:
def derivative(xy_coords):
    '''
    Parameters:
        xy_coords (np array): 0th column is x-coordinates, 1st column is y-coordinates
        
    Returns:
        Np array where 0th column is x-coordinates and nth column is nth forward discrete derivative
    '''
    sort = np.argsort(xy_coords[:,0]) 
    xy_coords = xy_coords[sort,:]
    x_coords = xy_coords[:,0]
    y_coords = xy_coords[:,1]
    deriv = [(y_coords[i]-y_coords[i-1])/(x_coords[i]-x_coords[i-1]) for i in range(1,len(xy_coords))]
    return np.column_stack((x_coords[:-1], deriv))

In [323]:
def kthderiv(xy_coords_extended, k, original_length):
    '''
    Parameters:
        xy_coords_extended (2-column np array): points to take derivatives of plus at least k predictions
        k (int): number of derivatives
        original_length (int): length of original list
    
    Returns:
        derivatives (numpy array): the ith column has the ith derivative of the list, up to the kth derivative
    '''
    
    derivatives = xy_coords_extended[:original_length, 0]
    for i in range(1,k+1):
        next_deriv = xy_coords_extended[:original_length + i, [0,-1]]
        for j in range(i):
            next_deriv = derivative(next_deriv)
        derivatives = np.column_stack((derivatives, next_deriv[:,1]))
    return derivatives

In [324]:
def holdout_variance_explained(pred, pred_length, y_pred):
    '''
    Parameters:
        pred (list): list of predictions
        pred_length (int): length of prediction
        y_pred (list): correct future values
        
    Returns:
        Percentage of variance explained for holdout
    '''
    y_pred = np.array(y_pred)
    score = np.sqrt(metrics.mean_squared_error(pred[-pred_length:],y_pred[-pred_length:]))
    variance_explained = ((y_pred[:-pred_length].std()-score)/y_pred[:-pred_length].std())*100
    return variance_explained

In [325]:
def variability(mylist):
    mylist = np.array(mylist)
    mylist = mylist - min(mylist)
    mylist = mylist/max(mylist)
    sum_square_differences = sum([(mylist[i] - mylist[i-1])**2 for i in range(1, len(mylist))])
    return (sum_square_differences/len(mylist))**0.5

In [326]:
# https://stackoverflow.com/questions/15785719/how-to-print-a-dictionary-line-by-line-in-python
# Prints out dictionaries nicely
def dumpclean(obj):
    if isinstance(obj, dict):
        for k, v in obj.items():
            if hasattr(v, '__iter__'):
                print(k)
                dumpclean(v)
            else:
                print('%s : %s' % (k, v))
    elif isinstance(obj, list):
        for v in obj:
            if hasattr(v, '__iter__'):
                dumpclean(v)
            else:
                print(v)
    else:
        print(obj)

# Tensorflow Code

In [405]:
# Creates a neural network with three hidden layers
seed = 5
def forecast(x, y, x_pred, dim, layers, neurons):
    '''
    Parameters:
        x (numpy array): each row is a single input, contains the appropriate derivatives
        y (numpy array): k x 1 array of all of the training labels
        x_pred (numpy array): array with row length the same as x, used as input for predictions
        dim (int): number of columns in x
        layers (int): number of layers (excluding output)
        neurons (int): total number of neurons
    
    Returns:
        pred: predicted values for x-coordinates in x_pred
    
    1. Creates a feedforward neural network where each layer is half the size of the previous one
    2. Trains the network based on x and y
    3. Returns network output when x_pred is put in the network
    '''
    
    tensorflow.random.set_seed(seed)
    
    model = Sequential()
    layer_size = 2**(layers - 1) * neurons//(2**(layers) - 1)
    model.add(Dense(layer_size, input_dim=dim, activation='relu'))
    
    for i in range(1,layers):
        layer_size = 2**(layers - i - 1) * neurons//(2**(layers) - 1)
        model.add(Dense(layer_size, activation='relu')) # Add hidden layer half the size of the previous one
        
    model.add(Dense(1)) # Output
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(x,y,verbose=0,epochs=512)
    pred = model.predict(x_pred) # generate predicted values (output)
    return pred

def nderiv_extrapolation_nonrecursive(x, y, x_pred, y_pred, n_derivs, layers, neurons): 
    '''
    Parameters:
        x (list or array): x-coordinates of known values
        y (list or array): known function values
        x_pred (list or array): original x-coordinates plus x-coordinates we want predictions for
        y_pred (list or array): original y-coordinates plus correct y-coordinates of future values
        n_derivs (int): number of derivatives to use
        layers (int): number of layers (excluding output)
        neurons (int): total number of neurons
        
    Returns:
        updated_pred: new predictions based on derivative regressors
    
    1. x_pred_extended is an extended version of x_pred with n_derivs extra elements
    2. Predicts function values in x_pred_extended
    3. Create an array, x_derivs, of regressors, where the ith column has the ith derivative of the original function
        - Array has x rows and n_derivs+1 columns.
    4. Create an array. x_pred_multi, of derivatives of all values, including the predicted values
        - Array has length x_pred and n_derivs+1 columns.
    5. Train the network using the array of regressors
    6. Predict the function values for x_pred_multi
    7. Return the predicted function values
    '''
    x = np.array(x)
    y = np.array(y)
    x_pred = np.array(x_pred)
    y_pred = np.array(y_pred)
    pred_length = len(x_pred) - len(x)
    
    if (n_derivs == 0):
        pred = forecast(x,y, x_pred, 1, layers, neurons)
        pred = np.squeeze(pred)
        return holdout_variance_explained(pred, pred_length, y_pred)
    
    pred_length = len(x_pred) - len(x)
    extension_step = np.mean([x_pred[i]-x_pred[i-1] for i in range(1,len(x_pred))])
    extension = [x_pred[-1] + (1 + i)*extension_step for i in range(n_derivs)]
    x_pred_extended = np.concatenate((x_pred, extension)) # extend x_pred by n_derivs to compensate for decrease in length from differentiating
    pred_extended = forecast(x, y, x_pred_extended, 1, layers, neurons) # forecast function values for x-coordinates in x_pred_extended
    pred_extended = list(np.squeeze(pred_extended)) # turn pred_extended into a list of floats
    xy_all = np.column_stack((x_pred_extended, np.concatenate((y, pred_extended[len(y):]))))
    x_pred_multi = kthderiv(xy_all, n_derivs, len(y) + pred_length)
    x_derivs = x_pred_multi[:len(y),:] # len(y) rows at beginning of x_pred_multi are the training set
    updated_pred = forecast(x_derivs, y, x_pred_multi, n_derivs+1, layers, neurons) # forecast based on added regressors
    updated_pred = np.squeeze(updated_pred)
    return holdout_variance_explained(updated_pred, pred_length, y_pred)

def nderiv_extrapolation_recursive(x, y, x_pred, y_pred, n_derivs, layers, neurons):
    '''
    This function generates a new derivative regressor each time the network is run. 
    The predictions from the kth run are used to create the kth derivative regressors and the kth derivatives for predicted values.
    The network is then trained again using the kth derivative regressors to get new predictions.
    Then the k+1th derivative regressors and the k+1th derivatives for predicted values are calculated.
    Repeat for however many derivatives are desired.
    
    Parameters:
        x (list or array): x-coordinates of known values
        y (list or array): known function values
        x_pred (list or array): original x-coordinates plus x-coordinates we want predictions for
        y_pred (list or array): original y-coordinates plus correct y-coordinates of future values
        n_derivs (int): number of derivatives to use
        layers (int): number of layers (excluding output)
        neurons (int): total number of neurons
        
    Returns:
        updated_pred: new predictions based on derivative regressors
    
    1. x_pred_extended is an extended version of x_pred with 1+2+...+n_derivs extra elements
    2. Predicts function values in x_pred_extended
    Loop:
        3. Create an array, x_derivs, of regressors, where the ith column has the ith derivative of the original function.
            - Array has x rows and n_derivs+1 columns.
        4. Create an array, x_pred_multi, of derivatives of all values, including the predicted values.
            - Each time, the number of rows decreases by a number equal to the current derivative being taken
        5. Train the network using the array of regressors
        6. Predict the function values for x_pred_multi
    End loop
    7. Return the predicted function values
    '''

    x = np.array(x)
    y = np.array(y)
    x_pred = np.array(x_pred)
    y_pred = np.array(y_pred)
    pred_length = len(x_pred) - len(x)
    
    if (n_derivs == 0):
        pred = forecast(x, y, x_pred, 1, layers, neurons)
        pred = np.squeeze(pred)
        return holdout_variance_explained(pred, pred_length, y_pred)
    
    extension_step = np.mean([x_pred[i]-x_pred[i-1] for i in range(1,len(x_pred))])
    extension_length = int(n_derivs*(n_derivs+1)/2)
    extension = [x_pred[-1] + extension_step*(1 + i) for i in range(extension_length)]
    x_pred_extended = np.concatenate((x_pred, extension))
    pred_extended = forecast(x, y, x_pred_extended, 1, layers, neurons)
    pred_extended = list(np.squeeze(pred_extended))
    
    for deriv in range(1, n_derivs + 1):
        extension_length = extension_length - deriv
        xy_all = np.column_stack((x_pred_extended, np.concatenate((y, pred_extended[len(y):]))))
        x_pred_multi = kthderiv(xy_all, deriv, len(y) + pred_length + extension_length)
        x_derivs = x_pred_multi[:len(y),:]
        x_pred_extended = x_pred_extended[:len(y) + pred_length + extension_length]
        pred_extended = forecast(x_derivs, y, x_pred_multi, deriv + 1, layers, neurons)
        pred_extended = list(np.squeeze(pred_extended))
    
    return holdout_variance_explained(pred_extended, pred_length, y_pred)

## Tensorflow example on y = sin(x)

In [ ]:
x = [i for i in range(200)]
y = [i*math.sin(i/200) for i in range(200)]
x_pred = [i for i in range(220)]
y_pred = [i*math.sin(i/200) for i in range(220)]
v_explained = nderiv_extrapolation_recursive(x, y, x_pred, y_pred, 2, 3, 2000)
print(v_explained)

# Prophet Code

In [204]:
# Use to test Prophet on plain functions
def create_df(x, y):
    df = pd.DataFrame()
    df['ds'] = pd.to_datetime(x, unit='D')
    df['y'] = y
    return df

In [205]:
def prophetpredict(training,horizon,horizon_freq,regressors,retail_data = True):
    '''
    Parameters:
        training (dataframe): ['ds'] contains dates, ['y'] contains values
        horizon (int): number of predictions
        horizon_freq (string): time interval between predictions
        regressors (dataframe): all values for regressors, including future regressors
        retail_data (bool): set to True when running on real data
    
    Returns:
        List of model outputs for known dates plus predictions
    '''
    data_length = len(training.index)

    if retail_data:
        m = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
        m.add_country_holidays(country_name='US')
        m.train_holiday_names
    else:
        m = Prophet(yearly_seasonality=False, daily_seasonality=False, weekly_seasonality=False)
    
    for col in regressors:
        m.add_regressor(col)
    all_input = pd.concat((training, regressors.head(data_length)), axis=1)
    m.fit(all_input)
    future = m.make_future_dataframe(periods=horizon, freq=horizon_freq)
    all_future_input = pd.concat((future, regressors.head(data_length+horizon)), axis=1)
    forecast = m.predict(all_future_input)
    predicteddf=forecast[['ds', 'yhat']]
    return list(predicteddf['yhat'])

In [18]:
def prophetnderivatives(df, horizon, horizon_freq, n_derivs, regressors, retail_data=True):
    '''
    Parameters:
        df (dataframe): ['ds'] contains all dates, ['y'] contains all values (train and test combined)
        horizon (int): length of test set/number of predictions
        horizon_freq (string): time interval between predictions
        n_derivs (int): number of derivative regressors to use
        regressors (dataframe): all values for regressors
        retail_data (bool): set to True when running on real data
        
    Returns:
        Percentage of variance explained for holdout
    '''
    data_length = len(df.index) - horizon
    
    if n_derivs == 0:
        pred = prophetpredict(df.head(data_length), horizon + n_derivs, horizon_freq, regressors, retail_data)
        return holdout_variance_explained(pred, horizon, list(df['y']))
        
    pred = prophetpredict(df.head(data_length), horizon + n_derivs, horizon_freq, pd.DataFrame(), retail_data)
    
    y_train = list(df['y'].head(data_length))
    
    col_names = ['derivative_' + str(i+1) for i in range(n_derivs)]
    future_derivatives = kthderiv(y_train + pred[data_length:], n_derivs, data_length + horizon)[:,1:]
    future_derivatives = pd.DataFrame(data=future_derivatives, columns=col_names)
    all_future_regressors = pd.concat((future_derivatives, regressors), axis=1)

    P = prophetpredict(df.head(data_length), horizon, horizon_freq, all_future_regressors, retail_data)
    variance_explained = holdout_variance_explained(P, horizon, list(df['y']))
    return variance_explained